IMPORT

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import math


from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000


#path = '/Koding_With_Kolesh/challenges/RMB_NOWCAST_APRIL/'
cpi = pd.read_csv('CPI_Historic_Values_Zindi_May_23.csv')
vehicles = pd.read_csv('Naamsa_Vehicle_Sales.csv')

PIVOTING THE DATA

In [2]:
cpi_pivot = cpi.pivot(index = 'Month', columns = 'Category', values = 'Value').reset_index()#changing from a long format to a wide format,
cpi_pivot['Month'] = pd.to_datetime(cpi_pivot['Month'])
cpi_pivot = cpi_pivot.sort_values("Month").reset_index(drop=True)

ADDING JUNE'S DATA MANUALLY FROM https://www.statssa.gov.za/publications/P0141/P0141June2023.pdf

In [3]:
date_str = '2023-06-30'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)
cpi_of_june = [110.9, 104.3, 99.6, 110.4, 118.3, 109.8, 110.8, 107.7, 105.4, 109.6, 105.3, 110.0, 112.3]

for i, col in enumerate(cpi_pivot.columns[1:]):
    cpi_pivot.at[17, col] = cpi_of_june[i]
cpi_pivot

,Month,Alcoholic beverages and tobacco,Clothing and footwear,Communication,Education,Food and non-alcoholic beverages,Headline_CPI,Health,Household contents and services,Housing and utilities,Miscellaneous goods and services,Recreation and culture,Restaurants and hotels,Transport
0,2022-01-31,100.0,100.2,99.8,100.0,100.9,100.2,100.1,100.4,100.0,100.6,100.2,101.2,98.9
1,2022-02-28,100.6,100.3,99.7,100.0,101.8,100.8,103.2,100.4,100.1,101.5,100.3,101.9,100.6
2,2022-03-31,102.8,100.5,99.7,104.4,102.4,101.8,103.7,100.8,100.5,101.9,100.4,103.0,103.7
3,2022-04-30,103.6,100.7,99.6,104.4,103.1,102.4,104.0,101.1,100.5,102.8,100.7,103.3,105.1
4,2022-05-31,104.4,100.9,99.9,104.4,105.3,103.1,104.3,101.8,100.6,102.9,101.0,103.8,105.9
5,2022-06-30,104.5,101.3,100.3,104.4,106.6,104.2,104.6,103.0,101.2,103.0,101.2,104.2,110.3
6,2022-07-31,104.4,101.6,99.4,104.4,107.8,105.8,104.1,103.3,103.1,103.4,102.3,104.6,115.6
7,2022-08-31,104.7,102.1,100.1,104.4,109.7,106.0,104.2,103.7,103.2,103.5,102.4,104.3,114.5
8,2022-09-30,105.6,102.5,99.9,104.4,110.3,106.1,104.0,104.5,103.7,103.8,102.5,106.2,112.0
9,2022-10-31,106.1,102.6,99.8,104.4,111.3,106.5,104.5,105.0,103.8,104.7,102.7,107.4,111.7


ADD THE july ROW

In [5]:
date_str = '2023-07-31'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)

FEATURE ENGINNERING

In [6]:
feats_to_lag = cpi_pivot.columns[1:].to_list()
for col in feats_to_lag:
    for i in range(1,5):
        cpi_pivot[f'prev_{i}_month_{col}'] = cpi_pivot[col].shift(i)
cpi_pivot

,Month,Alcoholic beverages and tobacco,Clothing and footwear,Communication,Education,Food and non-alcoholic beverages,Headline_CPI,Health,Household contents and services,Housing and utilities,Miscellaneous goods and services,Recreation and culture,Restaurants and hotels,Transport,prev_1_month_Alcoholic beverages and tobacco,prev_2_month_Alcoholic beverages and tobacco,prev_3_month_Alcoholic beverages and tobacco,prev_4_month_Alcoholic beverages and tobacco,prev_1_month_Clothing and footwear,prev_2_month_Clothing and footwear,prev_3_month_Clothing and footwear,prev_4_month_Clothing and footwear,prev_1_month_Communication,prev_2_month_Communication,prev_3_month_Communication,prev_4_month_Communication,prev_1_month_Education,prev_2_month_Education,prev_3_month_Education,prev_4_month_Education,prev_1_month_Food and non-alcoholic beverages,prev_2_month_Food and non-alcoholic beverages,prev_3_month_Food and non-alcoholic beverages,prev_4_month_Food and non-alcoholic beverages,prev_1_month_Headline_CPI,prev_2_month_Headline_CPI,prev_3_month_Headline_CPI,prev_4_month_Headline_CPI,prev_1_month_Health,prev_2_month_Health,prev_3_month_Health,prev_4_month_Health,prev_1_month_Household contents and services,prev_2_month_Household contents and services,prev_3_month_Household contents and services,prev_4_month_Household contents and services,prev_1_month_Housing and utilities,prev_2_month_Housing and utilities,prev_3_month_Housing and utilities,prev_4_month_Housing and utilities,prev_1_month_Miscellaneous goods and services,prev_2_month_Miscellaneous goods and services,prev_3_month_Miscellaneous goods and services,prev_4_month_Miscellaneous goods and services,prev_1_month_Recreation and culture,prev_2_month_Recreation and culture,prev_3_month_Recreation and culture,prev_4_month_Recreation and culture,prev_1_month_Restaurants and hotels,prev_2_month_Restaurants and hotels,prev_3_month_Restaurants and hotels,prev_4_month_Restaurants and hotels,prev_1_month_Transport,prev_2_month_Transport,prev_3_month_Transport,prev_4_month_Transport
0,2022-01-31,100.0,100.2,99.8,100.0,100.9,100.2,100.1,100.4,100.0,100.6,100.2,101.2,98.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-02-28,100.6,100.3,99.7,100.0,101.8,100.8,103.2,100.4,100.1,101.5,100.3,101.9,100.6,100.0,NaN,NaN,NaN,100.2,NaN,NaN,NaN,99.8,NaN,NaN,NaN,100.0,NaN,NaN,NaN,100.9,NaN,NaN,NaN,100.2,NaN,NaN,NaN,100.1,NaN,NaN,NaN,100.4,NaN,NaN,NaN,100.0,NaN,NaN,NaN,100.6,NaN,NaN,NaN,100.2,NaN,NaN,NaN,101.2,NaN,NaN,NaN,98.9,NaN,NaN,NaN
2,2022-03-31,102.8,100.5,99.7,104.4,102.4,101.8,103.7,100.8,100.5,101.9,100.4,103.0,103.7,100.6,100.0,NaN,NaN,100.3,100.2,NaN,NaN,99.7,99.8,NaN,NaN,100.0,100.0,NaN,NaN,101.8,100.9,NaN,NaN,100.8,100.2,NaN,NaN,103.2,100.1,NaN,NaN,100.4,100.4,NaN,NaN,100.1,100.0,NaN,NaN,101.5,100.6,NaN,NaN,100.3,100.2,NaN,NaN,101.9,101.2,NaN,NaN,100.6,98.9,NaN,NaN
3,2022-04-30,103.6,100.7,99.6,104.4,103.1,102.4,104.0,101.1,100.5,102.8,100.7,103.3,105.1,102.8,100.6,100.0,NaN,100.5,100.3,100.2,NaN,99.7,99.7,99.8,NaN,104.4,100.0,100.0,NaN,102.4,101.8,100.9,NaN,101.8,100.8,100.2,NaN,103.7,103.2,100.1,NaN,100.8,100.4,100.4,NaN,100.5,100.1,100.0,NaN,101.9,101.5,100.6,NaN,100.4,100.3,100.2,NaN,103.0,101.9,101.2,NaN,103.7,100.6,98.9,NaN
4,2022-05-31,104.4,100.9,99.9,104.4,105.3,103.1,104.3,101.8,100.6,102.9,101.0,103.8,105.9,103.6,102.8,100.6,100.0,100.7,100.5,100.3,100.2,99.6,99.7,99.7,99.8,104.4,104.4,100.0,100.0,103.1,102.4,101.8,100.9,102.4,101.8,100.8,100.2,104.0,103.7,103.2,100.1,101.1,100.8,100.4,100.4,100.5,100.5,100.1,100.0,102.8,101.9,101.5,100.6,100.7,100.4,100.3,100.2,103.3,103.0,101.9,101.2,105.1,103.7,100.6,98.9
5,2022-06-30,104.5,101.3,100.3,104.4,106.6,104.2,104.6,103.0,101.2,103.0,101.2,104.2,110.3,104.4,103.6,102.8,100.6,100.9,100.7,100.5,100.3,99.9,99.6,99.7,99.7,104.4,104.4,104.4,100.0,105.3,103.1,102.4,101.8,103.1

HANDLE MISSING DATA

In [7]:
cpi_pivot = cpi_pivot.drop(0)
cpi_pivot = cpi_pivot.bfill()

TRAIN AND VALIDATION

In [8]:
train = cpi_pivot[cpi_pivot['Month'] != "2023-07-31"]
test = cpi_pivot[cpi_pivot['Month'] == "2023-07-31"]

training_set = train[train['Month']!= '2023-06-30']
validation_set = train[train['Month']== '2023-06-30']

train.shape, test.shape, training_set.shape, validation_set.shape

((17, 66), (1, 66), (16, 66), (1, 66))

MODELING

In [9]:
target_cols = ['Alcoholic beverages and tobacco', 'Clothing and footwear',
       'Communication', 'Education', 'Food and non-alcoholic beverages',
       'Headline_CPI', 'Health', 'Household contents and services',
       'Housing and utilities', 'Miscellaneous goods and services',
       'Recreation and culture', 'Restaurants and hotels ', 'Transport']

#if you add additional data sources that have no value in the predicting month , drop it, now that you have their lags
features= [col for col in train.columns if col not in target_cols + ['Month']]


X_train = training_set[features]
y_train = training_set[target_cols]

X_val = validation_set[features]
y_val = validation_set[target_cols]

x_models = {}
lr_models = {}
y_pred = []
y_predx = []
scaler = MinMaxScaler()

#training
for target_col in target_cols:
    lr_model = LinearRegression()
    x_model = XGBRegressor()
    X_train_scaled = scaler.fit_transform(X_train)
    lr_model.fit(X_train_scaled, y_train[target_col])
    x_model.fit(X_train, y_train[target_col])
    lr_models[target_col] = lr_model
    x_models[target_col] = x_model
#validation
for target_col in target_cols:
    lr_model = lr_models[target_col]
    x_model = x_models[target_col]
    X_val_scaled = scaler.transform(X_val)
    y_pred_col = lr_model.predict(X_val_scaled)
    y_pred_colx = x_model.predict(X_val)
    y_pred.append(y_pred_col)
    y_predx.append(y_pred_colx)


#scoring

y_pred = np.array(y_pred).T
y_predx = np.array(y_predx).T
df = pd.DataFrame({'y_pred': y_pred.flatten(), 'y_val': y_val.values.flatten()})
dfx = pd.DataFrame({'y_pred': y_predx.flatten(), 'y_val': y_val.values.flatten()})
#calculate the rmse
rmse = np.sqrt(mean_squared_error(df['y_pred'], df['y_val']))
rmsex = np.sqrt(mean_squared_error(dfx['y_pred'], df['y_val']))
print(f'RMSE of Liner Regression: {rmse}') 
print(f'RMSE of XGB: {rmsex}') 

RMSE of Liner Regression: 1.7572439547650487
RMSE of XGB: 0.3753174210565328


In [10]:
X_train = train[features]
y_train = train[target_cols]

X_val = test[features]
y_val = test[target_cols]

x_models = {}
lr_models = {}
y_pred = []
y_predx = []
scaler = MinMaxScaler()

#training
for target_col in target_cols:
    lr_model = LinearRegression()
    x_model = XGBRegressor()
    X_train_scaled = scaler.fit_transform(X_train)
    lr_model.fit(X_train_scaled, y_train[target_col])
    x_model.fit(X_train, y_train[target_col])
    lr_models[target_col] = lr_model
    x_models[target_col] = x_model
#pridiction
for target_col in target_cols:
    lr_model = lr_models[target_col]
    x_model = x_models[target_col]
    X_val_scaled = scaler.transform(X_val)
    y_pred_col = lr_model.predict(X_val_scaled)
    y_pred_colx = x_model.predict(X_val)
    y_pred.append(y_pred_col)
    y_predx.append(y_pred_colx)

y_pred = np.array(y_pred).T
y_predx = np.array(y_predx).T
print(f'pridiction of Liner Regression: {y_pred}') 
print(f'prediction of XGB: {y_predx}') 

pridiction of Liner Regression: [[110.85578063 104.65265584  98.27550679 109.39268781 119.25251737
  111.07716113 110.6279201  106.82637783 107.39454682 110.50461312
  105.94195778 110.27982706 116.08566678]]
prediction of XGB: [[110.89876  104.298836  99.59974  110.39956  118.298836 109.79871
  110.79841  107.69692  105.39884  109.59883  105.29846  110.39889
  112.36836 ]]


SUBMITION

In [11]:
def prepSub(y_pred:list, target_cols: list, test, prefix:str):
    sub_df = pd.DataFrame(y_pred, columns=target_cols)
    sub_df['Month'] = test['Month']
    
    sub_df.set_index('Month', inplace=True)
    sub_df.columns = [prefix+'_' + col.lower().replace('_', ' ').strip() for col in sub_df.columns]
    sub_df.rename(columns= {f"{prefix}_headline cpi": f"{prefix}_headline CPI"}, inplace=True)
    
    sub_df = pd.melt(sub_df.reset_index(), id_vars= ['Month'], var_name= 'ID', value_name= 'Value')
    
    return sub_df[['ID', 'Value']]

In [12]:
sub = prepSub(y_pred, target_cols, test, 'July') 
sub.to_csv('out/base_liner_with_lag_historic_only.csv', index=False)